# A Primer on the FITS Data Format<img src='images/logo.png' width="300" height="300" align='right'>


Astronomers use a particular file format called FITS. There are many reasons for this, but let's not worry about that right now. Following is a quick primer on the FITS file format taken (almost verbatim) from [this NASA website](https://fits.gsfc.nasa.gov/fits_primer.html).

FITS (Flexible Image Transport System) is the data format most widely used within astronomy for transporting, analyzing, and archiving scientific data files. FITS is much more than just another image format (such as JPG or GIF) and is primarily designed to store scientific data sets consisting of multidimensional arrays (images) and 2-dimensional tables organized into rows and columns of information.

## HDUs

A FITS file is comprised of segments called Header/Data Units (HDUs), where the first HDU is called the "Primary HDU," or "Primary Array." The primary data array can contain a 1-999 dimensional array of 1, 2 or 4 byte integers or 4 or 8 byte floating point numbers using IEEE representations. A typical primary array could contain a 1-D spectrum, a 2-D image, or a 3-D data cube.
Any number of additional HDUs may follow the primary array. These additional HDUs are referred to as FITS "extensions." Three types of standard extensions are currently defined:

<ul>
<li>Image Extensions contain a 0-999 dimensional array of pixels, similar to a primary array 
(header begins with XTENSION = 'IMAGE ')</li>
<li>ASCII Table Extensions store tabular information with all numeric information stored in ASCII formats. While ASCII tables are generally less efficient than binary tables, they can be made relatively human readable and can store numeric information with essentially arbitrary size and accuracy (e.g., 16 byte reals). 
(header begins with XTENSION = 'TABLE ')</li>
<li>Binary Table Extensions store tabular information in a binary representation. Each cell in the table can be an array but the dimensionality of the array must be constant within a column. The strict standard supports only one-dimensional arrays, but a convention to support multi-dimensional arrays is widely accepted. 
(header begins with XTENSION = 'BINTABLE')</li>
</ul>

In addition to the structures described above, there is one other type of FITS HDU called "Random Groups" that is almost exclusively used for applications in radio interferometry. The random groups format should not be used for other types of applications.

## Header Units

Every HDU consists of an ASCII formatted "Header Unit" followed by an optional "Data Unit." Each header or data unit is a multiple of 2880 bytes long. If necessary, the header or data unit is padded out to the required length with ASCII blanks or NULLs depending on the type of unit.
Each header unit contains a sequence of fixed-length 80-character keyword records which have the general form:

`KEYNAME = value / comment string`

The keyword names may be up to 8 characters long and can only contain uppercase letters A to Z, the digits 0 to 9, the hyphen, and the underscore character. The keyword name is (usually) followed by an equals sign and a space character in columns 9 and 10 of the record, followed by the value of the keyword which may be either an integer, a floating point number, a complex value (i.e., a pair of numbers), a character string (enclosed in single quotes), or a Boolean value (the letter T or F). Some keywords, (e.g., COMMENT and HISTORY) are not followed by an equals sign and in that case columns 9 - 80 of the record may contain any string of ASCII text.

Each header unit begins with a series of required keywords that specify the size and format of the following data unit. A 2-dimensional image primary array header, for example, begins with the following keywords:

`SIMPLE  =                    T / file conforms to FITS standard
BITPIX  =                   16 / number of bits per data pixel
NAXIS   =                    2 / number of data axes
NAXIS1  =                  440 / length of data axis 1
NAXIS2  =                  300 / length of data axis 2`

The required keywords may be followed by other optional keywords to describe various aspects of the data, such as the date and time of the observation. COMMENT or HISTORY keywords are also frequently added to further document the contents of the data file.
The last keyword in the header is always the "END" keyword which has blank value and comment fields. The header is padded with additional blank records if necessary so that it is a multiple of 2880 bytes (equivalent to 36 80-byte keywords) long. Note that the header unit may only contain ASCII text characters ranging from hexadecimal 20 to 7E); non-printing ASCII characters such as tabs, carriage-returns, or line-feeds are not allowed anywhere within the header unit.

## Data Units

The data unit, if present, immediately follows the last 2880-byte block in the header unit. Note that the data unit is not required, so some HDUs only contain the header unit.
The image pixels in a primary array or an image extension may have one of 5 supported data types:

<ul>
<li>8-bit (unsigned) integer bytes</li>
<li>16-bit (signed) integers</li>
<li>32-bit (signed) integers</li>
<li>32-bit single precision floating point real numbers</li>
<li>64-bit double precision floating point real numbers</li>
</ul>

A 64-bit integer datatype has also been proposed and is currently in experimental use. Unsigned 16-bit and 32-bit integers are supported by subtracting an offset from the raw pixel values (e.g., 32768 or 2<sup>15</sup>) is subtracted from each unsigned 16-bit integer pixel value to shift the values into the range of a signed 16-bit integer) before writing them to the FITS file. This offset is then added to the pixels when reading the FITS image to restore the original values.

The other 2 types of standard extensions, ASCII tables and binary tables, contain tabular information organized into rows and columns. Generally speaking, binary tables are more compact and are faster to read and write than than ASCII tables because the numeric entries are stored in machine readable binary representation.

All the entries within a given column of an ASCII or Binary table extension have the same datatype. The allowed data formats for an ASCII table column are: integer, single or double precision floating point value, or character string. Binary table extensions also support logical (T/F), bit, and complex data formats.

Each entry, or field, in an ASCII table may only contain 1 scalar value. Binary tables are more flexible and allow N-dimensional arrays of of data (either fixed length or variable length) to be stored within each field. Variable-length arrays are implemented by storing a pointer in the field of the table which defines the length and byte offset to the start of the data array which is located in the "heap" area that follows the table proper. The variable-length-arrays convention is not strictly part of the FITS standard but is widely used.

# The FITS header

This tutorial describes how to read in, edit a FITS header, and then write 
it back out to disk. For this example we're going to change the `OBJECT` 
keyword.

This tutorial uses [astropy.io.fits](http://docs.astropy.org/en/latest/io/fits/index.html), 
which was formerly released separately as `pyfits`. If you have used 
`pyfits` to manipulate FITS files then you may already be familiar 
with the features and syntax of the package. We start by importing
the subpackage into our local namespace, and allows us to access the
functions and classes as `fits.name_of_function()`. For example, 
to access the `getdata()` function, we _don't_ have to do 
`astropy.io.fits.getdata()` and can instead simple use `fits.getdata()`.
You may run across old documentation or tutorials that use the name 
`pyfits`. Such examples will begin with `import pyfits` and then 
the command `fits.getdata()` (for example) would be written as 
`pyfits.getdata()`.

In [1]:
from astropy.io import fits

ModuleNotFoundError: No module named 'astropy'

``astropy.io.fits`` provides a lot of flexibility for reading FITS 
files and headers, but most of the time the convenience functions are
the easiest way to access the data. ``fits.getdata()`` reads just the 
data from a FITS file, but with the `header=True` keyword argument will
also read the header. 

In [ ]:
data,header = fits.getdata("data/input_file.fits", header=True)

: 

There is also a dedicated function for reading just the 
header:

In [ ]:
hdu_number = 0
fits.getheader('data/input_file.fits', hdu_number)

: 

but `getdata()` can get both the data and the header, so it is a useful 
command to remember. Since the primary HDU of a FITS file must contain image data, 
the data is now stored in a ``numpy`` array. The header is stored in an 
object that acts like a standard Python dictionary. 

In [ ]:
print(type(data))
print(header["NAXIS"])

: 

Now let's change the header to give it the correct object:

In [ ]:
header['OBJECT'] = "M31"

: 

Finally, we have to write out the FITS file. Again, the convenience 
function for this is the most useful command to remember:

In [ ]:
fits.writeto('data/output_file.fits', data, header)

: 

That's it; you're done!

Two common more complicated cases are worth mentioning (but if your needs 
are much more complex, you should consult the full documentation). 

The first complication is that the FITS file you're examining and 
editing might have multiple HDU's (extensions), in which case you can 
specify the extension like this:

In [ ]:
data,header = fits.getdata("data/input_file.fits", ext=0, header=True)

: 

This will get you the data and header associated with the index=0 extension 
in the FITS file. Without specifying a number, getdata() will get the 
0th extension (equivalent to saying `ext=0`). This file happens to only have a 0th index, but some files have more than one

Another useful tip is if you want to overwrite an existing FITS 
file. By default, writeto() won't let you do this, so you need to 
explicitly give it permission using the `overwrite` keyword argument:

In [ ]:
fits.writeto('data/output_file.fits', data, header, overwrite=True)

: 

## Exercise

Read in the file you just wrote, and add three header keywords:

1. 'RA' for the Right Ascension of M31
2. 'DEC' for the Declination of M31
3. 'RADECSRC' with text indicating where you found the RA/Dec (web URL, textbook name, your photographic memory, etc.).

then write the updated header back out to a new file. 

: 